In [1]:
import os, glob
import rasterio
import numpy as np
import geopandas as gpd
import rasterio.features
from tqdm.notebook import tqdm

In [2]:
def create_mask_all(img_path, shp_path, list_name_object, property_name, out_path_mask):
    """
        property_name: day la ten truong ma chua cac doi tuong
    """
    with rasterio.open(img_path) as src:
        meta = src.meta
        height, width = src.height, src.width
        tr = src.transform
        crs_img = src.crs
    df = gpd.read_file(shp_path)
    df['valu'] = 0
    # check epsg
    if df.crs.to_string() != crs_img.to_string():
        df = df.to_crs(epsg=str(crs_img.to_epsg()))

    # check so luong class
    list_object_in_field = np.unique(df[property_name])
    if set(list_object_in_field).issubset(list_name_object):
        i = 0
        for class_name in list_name_object:
            i+=1
            if class_name in list_object_in_field:
                df.loc[df[property_name] == class_name, 'valu']= i
            else:
                continue

        shapes = df[['geometry', 'valu']]
        shapes = list(map(tuple, shapes.values))
        mask = rasterio.features.rasterize(shapes, out_shape=(height, width), transform=tr)
        meta.update({'count': 1, 'nodata': 0})
        with rasterio.open(out_path_mask, 'w', **meta) as dst:
            dst.write(np.array([mask]))   
    else:
        print(list_object_in_field)
        print(list_name_object)
        print(1)
        pass

    


## Chay 1 Anh

In [3]:
""" chay 1 anh """
img_path = r'/home/skm/SKM16/Work/OpenLand/Image/img_origin/visual_c57a04cf-cbf1-4611-a74d-e6cb433025a3/files/SkySatCollect/20220814_103841_ssc10_u0002/visual/20220814_103841_ssc10_u0002_visual.tif'
shp_path = r'/home/skm/SKM16/Work/OpenLand/Lables/20220814_103841_ssc10_u0002_visual.shp'
out_path_mask = r"/home/skm/SKM16/Tmp/tmp_xx.tif"
list_name_object = ['Buildup', 'Openland', 'Road', 'Water']
property_name = 'Class_Name'
create_mask_all(img_path, shp_path, list_name_object, property_name, out_path_mask)

## Chay all Folder

In [5]:
dir_img = r'/home/skm/SKM16/Work/OpenLand/all_tif'
dir_shape = r'/home/skm/SKM16/Work/OpenLand/Lables'
list_name_object = ['Buildup', 'Openland', 'Road', 'Water']
property_name = 'Class_Name'
# out_dir_mask = dir_img + '_mask_all_class'
out_dir_mask = r"/home/skm/SKM16/Work/OpenLand/3_dichHistogram/Image_allLabel"

os.makedirs(out_dir_mask, exist_ok=True)

list_fp_shp = glob.glob(os.path.join(dir_shape, '*.shp'))
list_fp_tif = glob.glob(os.path.join(dir_img, '*.tif'))

with tqdm(total=len(list_fp_shp)*len(list_fp_tif)) as pbar:
    for fp_shp in list_fp_shp:
        name_f = os.path.basename(fp_shp)[:-4]
        for fp_tif in list_fp_tif:
            name_img = os.path.basename(fp_tif)
            if name_f in fp_tif:
                out_path_mask = os.path.join(out_dir_mask, name_img)
                create_mask_all(fp_tif, fp_shp, list_name_object, property_name, out_path_mask)
            else:
                pbar.update(1)
                continue
            pbar.update(1)

  0%|          | 0/644 [00:00<?, ?it/s]

/home/skm/anaconda3/envs/geoai/lib/python3.7/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/home/skm/anaconda3/envs/geoai/lib/python3.7/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  aout[:] = out
/home/skm/anaconda3/envs/geoai/lib/python3.7/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/home/skm/anaconda3/envs/geoai/lib/python3.7/site-packages/geopandas/array.py:93:

## Chuyen mask quan tam ve 255

In [1]:
import glob
import rasterio
import numpy as np
from tqdm.notebook import tqdm

dir_mask_all_class = r"/home/skm/SKM16/Work/OpenLand/3_dichHistogram/Image_allLabel/MASK_ALL_OK"
# dir_mask_all_class = out_dir_mask

class_interests = [4]
# name = "buildUp"
# out_mask_255 = dir_mask_all_class + f'interest_class_{name}'

out_mask_255 = r"/home/skm/SKM16/Work/OpenLand/3_dichHistogram/Training_Water/Img_original_Water_mask"
os.makedirs(out_mask_255, exist_ok=True)

list_fp = glob.glob(os.path.join(dir_mask_all_class, '*.tif'))
for fp in tqdm(list_fp, desc = 'Progress Bar convert 255: '):
    fp_out = os.path.join(out_mask_255, os.path.basename(fp))
    with rasterio.open(fp) as src:
        meta = src.meta
        mask = src.read()
    for class_interest in class_interests:
        mask[mask != class_interest] = 0
    if np.any(mask):
        mask[mask == class_interest] = 255
        with rasterio.open(fp_out, 'w', **meta) as dst:
            dst.write(mask)

Progress Bar convert 255:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
# check dtype
# dir_mask_all_class = out_mask_255
dir_mask_all_class = r"/home/skm/SKM16/Work/OpenLand/3_dichHistogram/Training_Road/Img_original_Road_mask"
list_fp = glob.glob(os.path.join(dir_mask_all_class, '*.tif'))
for fp in list_fp:
    with rasterio.open(fp) as src:
        meta = src.meta
        print(meta['dtype'])

uint8
uint8
uint8
uint8
uint8
uint8


In [7]:
import rasterio
import numpy as np

dir_mask_all_class = r"/home/skm/SKM16/Work/OpenLand/3_dichHistogram/Image_allLabel/MASK_ALL_OK/20220814_103841_ssc10_u0002_visual.tif"
with rasterio.open(dir_mask_all_class) as src:
        meta = src.meta
        mask = src.read()
a = np.where(mask == 1)
b = np.where(mask == 10)

In [17]:
a.size

AttributeError: 'tuple' object has no attribute 'size'

In [9]:
b

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [11]:
len(b)

3

In [15]:
type(a)

tuple

In [14]:
type(b)

tuple

(array([0, 0, 0, ..., 0, 0, 0]),
 array([  754,   755,   755, ..., 26733, 26733, 26733]),
 array([ 3659,  3655,  3656, ..., 13401, 13402, 13403]))